In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# CLEANING DATA
# read csv files
honey_dataset = pd.read_csv("honey.csv")
pollution_dataset = pd.read_csv("pollution_us_2000_2016.csv")

In [3]:
# show the sum of null in each column (flagging)
# print(pollution_dataset.isnull().sum())
# print(honey_dataset.isna().sum())

# Missing values
# pollution us - S02 AQI 872907, CO AQI 873323

# fill missing values with mean values (filling)
mean = pollution_dataset["SO2 AQI"].mean()
mean2 = pollution_dataset["CO AQI"].mean()

pollution_dataset["SO2 AQI"].fillna(mean, inplace=True)
pollution_dataset["CO AQI"].fillna(mean2, inplace=True)

# No duplicates for honey and pollution datasets
# duplicates = pollution_dataset.duplicated(keep=False).sum()

# Drop data that is below 2000 and above 2016
# honey dataset
honey_dataset["year"] = honey_dataset["year"].astype("int")
honey_index = honey_dataset[(honey_dataset["year"] < 2000) | (honey_dataset["year"] > 2016)].index
honey_dataset = honey_dataset.drop(honey_index)

# pollution dataset
pollution_dataset["Date Local"] = pd.to_datetime(pollution_dataset["Date Local"])
pollution_dataset["Year"] = pollution_dataset["Date Local"].dt.year
pollution_dataset["Year"] = pollution_dataset["Year"].astype("int")

# sort by state name
honey_dataset.sort_values(by="state")
pollution_dataset.sort_values(by="State")

# Group by state & year
# honey
honey_stats = {"numcol": "mean"}
honey_dataset = honey_dataset.groupby(["year", "state"]).agg(honey_stats).reset_index()

# pollution
pollution_stats = {"NO2 AQI": "mean", "O3 AQI": "mean", "SO2 AQI": "mean", "CO AQI": "mean"}
pollution_dataset = pollution_dataset.groupby(["Year", "State"]).agg(pollution_stats).reset_index()

In [4]:
# Used to check the amount of duplicates within the dataframe based on the state column
temp_honey = honey_dataset.pivot_table(columns='state', aggfunc='size').reset_index()

#Rename the 0 column to count
temp_honey = temp_honey.rename(columns={0: 'count'})

#Locates and store the states which have a count lower than 17 
drop_honey_values = temp_honey.loc[temp_honey['count'] < 17]

#Take out states which are present within drop_honey_values from the honey_dataset
honey_dataset = honey_dataset[~honey_dataset.state.isin(drop_honey_values.state)]


# Used to check the amount of duplicates within the dataframe based on the state column
temp_pollution = pollution_dataset.pivot_table(columns='State', aggfunc='size').reset_index()

#Rename the 0 column to count
temp_pollution = temp_pollution.rename(columns={0: 'count'})

#Locates and store the states which have a count lower than 17
drop_pollution_values = temp_pollution.loc[temp_pollution['count'] < 17]

#Take out states which are present within drop_pollution_values from the pollution_dataset
pollution_dataset = pollution_dataset[~pollution_dataset.State.isin(drop_pollution_values.State)]

#Finds the common columns between the 2 dataframes based on state
common_cols = [col for col in set(honey_dataset.state).intersection(pollution_dataset.State)]

#Takes out states which are not in the common_cols from honey_dataset
honey_dataset = honey_dataset[honey_dataset.state.isin(common_cols)]

#Takes out states which are not in the common_cols from pollution_dataset
pollution_dataset = pollution_dataset[pollution_dataset.State.isin(common_cols)]

In [21]:
honey_dataset = honey_dataset.reset_index(drop=True)
pollution_dataset = pollution_dataset.reset_index(drop=True)

In [22]:
#Convert the dataframes into csv
honey_dataset.to_csv('cleaned_honey.csv')
pollution_dataset.to_csv('cleaned_pollution.csv')